# Medical Chatbot Project

## Libraries

In [14]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [41]:
from dotenv import load_dotenv

load_dotenv()

True

## Vector Database

In [17]:
# Extract Data From PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [18]:
# Load the data
extracted_data = load_pdf("data/")

### Text Chunks

In [19]:
# Create text chunks
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 20) # Select the size of characters of each chunk
    text_chunks = text_splitter.split_documents(data)

    return text_chunks


In [20]:
text_chunks = text_split(extracted_data)

In [21]:
len(text_chunks)

2565

### Embbeding Model

In [22]:
# Dowload Embedding Model
def download_emb():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_emb()


In [23]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Test of Embedding

In [24]:
query_result = embeddings.embed_query("Hello World")
query_result, len(query_result)

([-0.03447727486491203,
  0.031023237854242325,
  0.006734995637089014,
  0.026108987629413605,
  -0.039362020790576935,
  -0.1603025197982788,
  0.06692394614219666,
  -0.006441484205424786,
  -0.04745049774646759,
  0.014758791774511337,
  0.07087524980306625,
  0.05552758276462555,
  0.01919332891702652,
  -0.026251375675201416,
  -0.010109515860676765,
  -0.02694043703377247,
  0.0223073810338974,
  -0.022226618602871895,
  -0.1496926099061966,
  -0.017493048682808876,
  0.0076762172393500805,
  0.054352305829524994,
  0.003254448063671589,
  0.03172590956091881,
  -0.0846213772892952,
  -0.02940598502755165,
  0.0515955314040184,
  0.0481240339577198,
  -0.0033148115035146475,
  -0.058279164135456085,
  0.04196930676698685,
  0.022210676223039627,
  0.1281888335943222,
  -0.02233891747891903,
  -0.011656291782855988,
  0.06292839348316193,
  -0.03287632763385773,
  -0.09122605621814728,
  -0.03117537684738636,
  0.05269954726099968,
  0.047034818679094315,
  -0.0842030867934227,
 

In [25]:
import os
Pinecone(api_key=os.environ.get('PINECONE_API_KEY'),
              environment=os.environ.get('PINECONE_API_ENV'))

index_name="medical-chatbot"
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [26]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What is Drug Desing"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='104 Bioinformatics Tools in Drug Development\ndrug-likeness. It principally means the identification and removal of can-\ndidate molecules that are doubtful to endure the advanced phases of dis-\ncovery and development. Moreover, bioinformatics is also putting the base'), Document(page_content='104 Bioinformatics Tools in Drug Development\ndrug-likeness. It principally means the identification and removal of can-\ndidate molecules that are doubtful to endure the advanced phases of dis-\ncovery and development. Moreover, bioinformatics is also putting the base'), Document(page_content='time [36]. Numerous approaches and software solutions have been intro-\nduced [37]. But de novo  design has not seen a widespread use in drug discov-\nery. This is at least partially related to the generation of compounds, which are')]


## Large Langague Model

### Prompt Desing

In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [28]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [42]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':2048,
                          'temperature':0.8})


In [43]:
retriever = docsearch.as_retriever(search_kwargs={'k': 2})
docs = retriever.get_relevant_documents('Drug Desing')

In [44]:
len(docs)

2

In [45]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [46]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

KeyboardInterrupt: 